In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import scipy
from itertools import combinations
from scipy.stats import pearsonr, spearmanr
from statsmodels.sandbox.stats.multicomp import multipletests 

%pylab inline

import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


# Анализ эффективности удержания

В этом задании вам предлагается проанализировать данные одной из американских телекоммуникационных компаний о пользователях, которые потенциально могут уйти.

Измерены следующие признаки:

- state — штат США
- account_length — длительность использования аккаунта
- area_code — деление пользователей на псевдорегионы, использующееся в телекоме
- intl_plan — подключена ли у пользователя услуга международного общения
- vmail_plan — подключена ли у пользователя услуга голосовых сообщений
- vmail_message — количество голосых сообщений, который пользователь отправил / принял
- day_calls — сколько пользователь совершил дневных звонков
- day_mins — сколько пользователь проговорил минут в течение дня
- day_charge — сколько пользователь заплатил за свою дневную активность
- eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности
- night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности
- intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения
- custserv_calls — сколько раз пользователь позвонил в службу поддержки
- treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа)
- mes_estim — оценка интенсивности пользования интернет мессенджерами
- churn — результат оттока: перестал ли абонент пользоваться услугами оператора

In [2]:
df = pd.read_csv('churn_analysis.csv')
df.head(5)

,Unnamed: 0,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,0,KS,128,415,no,yes,25,265.1,110,45.07,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,1,OH,107,415,no,yes,26,161.6,123,27.47,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,2,NJ,137,415,no,no,0,243.4,114,41.38,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,3,OH,84,408,yes,no,0,299.4,71,50.90,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,4,OK,75,415,yes,no,0,166.7,113,28.34,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 23 columns):
Unnamed: 0        3333 non-null int64
state             3333 non-null object
account_length    3333 non-null int64
area_code         3333 non-null int64
intl_plan         3333 non-null object
vmail_plan        3333 non-null object
vmail_message     3333 non-null int64
day_mins          3333 non-null float64
day_calls         3333 non-null int64
day_charge        3333 non-null float64
eve_mins          3333 non-null float64
eve_calls         3333 non-null int64
eve_charge        3333 non-null float64
night_mins        3333 non-null float64
night_calls       3333 non-null int64
night_charge      3333 non-null float64
intl_mins         3333 non-null float64
intl_calls        3333 non-null int64
intl_charge       3333 non-null float64
custserv_calls    3333 non-null int64
treatment         3333 non-null int64
mes_estim         3333 non-null float64
churn             3333 non-null obje

## Задания
### Хи-квадрат
Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора.

Для этого мы воспользуемся критерием хи-квадрат. Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn. Для каждой такой таблицы 2x2 применить критерий хи-квадрат можно с помощью функции
```
stats.chi2_contingency(subtable, correction=False)
```
Заметьте, что, например, (AZ, HI) и (HI, AZ) — это одна и та же пара. Обязательно выставьте correction=False (о том, что это значит, вы узнаете из следующих вопросов).

Сколько достигаемых уровней значимости оказались меньше, чем $\alpha=0.05$?

In [4]:
df_treatment_1 = df[df.treatment == 1]
states = df_treatment_1.state.unique()
states_crosstab = pd.crosstab(df_treatment_1.state, df_treatment_1.churn)  

In [5]:
def print_states_chi_pvals(correction):
    states_combs = combinations(states, 2)
    states_churn_p = []
    for comb in states_combs:
        subtable = states_crosstab.loc[comb,:]
        states_churn_p.append(stats.chi2_contingency(subtable, correction=correction)[1])
    alpha = 0.05
    states_churn_p = np.array(states_churn_p)
    print('Среднее ',states_churn_p.mean())
    print('У стольких пар значения pvalue меньше', alpha, ':', sum(states_churn_p < alpha))

In [6]:
print_states_chi_pvals(False)

Среднее  0.5018273798739158
У стольких пар значения pvalue меньше 0.05 : 34


### Утверждения

Какие проблемы Вы видите в построении анализа из первого вопроса? Отметьте все верные утверждения.

- ~~Хи-квадрат используется для того, чтобы сравнить выборку с некоторым воздействием (treatment) и выборку без этого воздействия (control). Мы же в первом задании сравнивали штаты, используя данные только control группы. Для данных только из control группы использование xи-квадрат неправомерно.~~

- ~~Анализ нужно было начинать с применения xи-квадрат к таблице сопряженности, в которой присутствовали сразу все возможные штаты. Достигаемой уровень значимости такой проверки = 0.7, что дает нам гарантию, что нет ни одной пары штатов, в которых отличие в соотношениях ушедших и оставшихся клиентов статистически значимо.~~

- ~~Поправку на множественную проверку здесь применять нельзя — она используется только для группы критериев, проверяющих равенство средних (типа t-критерия). Критерий xи-квадрат не принадлежит этому семейству, поэтому поправка не нужна.~~

- Интерпретация числа достигаемых уровней значимости, меньших $\alpha=0.05$, некорректна, поскольку не сделана поправка на множественную проверку гипотез.

- Применение критерия xи-квадрат для этих данных не обосновано, потому что не выполняются условия, при которых этот критерий дает правильные результаты.

### Портим результаты для Хи- квадрат
В основе критерия xи-квадрат лежит предположение о том, что если верна нулевая гипотеза, то __дискретное__ биномиальное распределение данных по клеткам в таблице сопряженности может быть аппроксимировано с помощью __непрерывного__ распределения xи-квадрат. Однако точность такой аппроксимации существенно зависит от суммарного количества наблюдений и их распределения в этой таблице (отсюда и ограничения при использовании критерия xи-квадрат).

Одним из способов коррекции точности аппроксимации является поправка Йетса на непрерывность. Эта поправка заключается в вычитании константы 0.5 из каждого модуля разности наблюденного $O_i$ и ожидаемого $E_i$ значений, то есть, статистика с такой поправкой выглядит так: 

$$χ_{Yates}^2=∑_{i=1}^N\frac{(|Oi−Ei|−0.5)^2}{E_i}.$$

Такая поправка, как несложно догадаться по формуле, как правило, уменьшает значение статистики $χ^2$, то есть увеличивает достигаемый уровень значимости.

Эта поправка обычно используется для таблиц сопряженности размером 2x2 и для небольшого количества наблюдений. Такая поправка, однако, не является серебрянной пулей, и часто критикуется за то, что статистический критерий при ее использовании становится слишком консервативным, то есть часто не отвергает нулевую гипотезу там, где она неверна (совершает ошибку II рода).

Полезно знать, что эта поправка часто включена по умолчанию (например, в функции scipy.stats.chi2_contingency) и понимать ее влияние на оценку достигаемого уровня значимости.

Проведите те же самые сравнения, что и в вопросе №1, только с включенной коррекцией:
```
scipy.stats.chi2_contingency(subtable, correction=True)
```
и сравните полученные результаты, отметив все верные варианты. 

In [7]:
print_states_chi_pvals(True)

Среднее  0.6640566382051047
У стольких пар значения pvalue меньше 0.05 : 0


- ~~Точный критерий Фишера точно также, как и критерий xи-квадрат, нельзя использовать, если наблюдений < 40 и если ожидаемое значение меньше 5 больше чем в 20% ячейках.~~
- Количество достигаемых уровней значимости, меньших, чем 0.05, в точности равно нулю. То есть поправка увеличила достигаемые уровни значимости настолько, что больше ни одно из значений достигаемого уровня значимости не попадает в диапазон от 0 до 0.05.
- ~~Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем меньшие, чем xи-квадрат без поправки~~
- ~~Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем большие, чем xи-квадрат с поправкой Йетса~~
- Достигаемые уровни значимости на наших данных, полученные с помощью критерия xи-квадрат с поправкой Йетса, в среднем получаются больше, чем соответствующие значения без поправки.

### Фишер
Что если у нас мало данных, мы не хотим использовать аппроксимацию дискретного распределения непрерывным и использовать сомнительную поправку, предположения критерия xи-квадрат не выполняются, а проверить гипотезу о том, что данные принадлежат одному распределению, нужно ?

В таком случае прибегают к так называемому точному критерию Фишера. Этот критерий не использует приближений и в точности вычисляет значение достигаемого уровня значимости используя комбинаторный подход.

Пусть у нас есть таблица сопряженности 2x2:

k|Группа 1|Группа 2| $Σ$
--- | --- | ---|---
Воздействие 1|$a$|$b$|$a+b$
Воздействие 2|$c$|$d$|$c+d$
$Σ$|$a+c$|$b+d$|$n = a + b + c + d$

Тогда вероятность получить именно такие $a,b,c,d$ при фиксированных значениях сумм по строкам и по столбцам) задается выражением

сть получить именно такие $a,b,c,d$ при фиксированных значениях сумм по строкам и по столбцам) задается выражением

$$p={\frac {\displaystyle {{a+b} \choose {a}}\displaystyle {{c+d} \choose {c}}}{\displaystyle {{n} \choose {a+c}}}}={ \dfrac {(a+b)!~(c+d)!~(a+c)!~(b+d)!}{a!~~b!~~c!~~d!~~n!}}.$$

В числителе этой дроби стоит суммарное количество способов выбрать $a$ и $c$ из $a +b$ и $c + d$ соответственно. А в знаменателе — количество способов выбрать число объектов, равное сумме элементов первого столбца $a + c$ из общего количества рассматриваемых объектов $n$.

Чтобы посчитать достигаемый уровень значимости критерия Фишера, нужно перебрать все возможные значения $a, b, c, d$ в клетках этой таблицы так, чтобы построковые и постолбцовые суммы не изменились. Для каждого такого набора $a,b,c,d$ нужно вычислить значение $p_i$ по формуле выше и просуммировать все такие значения $p_i$, которые меньше или равны $p$, которое мы вычислили по наблюдаемым значениям $a,b,c,d$.

Понятно, что такой критерий вычислительно неудобен в силу большого количества факториалов в формуле выше. То есть даже при небольших выборках для вычисления значения этого критерия приходится оперировать очень большими числами. Поэтому данным критерием пользуются обычно только для таблиц 2x2, но сам критерий никак не ограничен количеством строк и столбцов, и его можно построить для любой таблицы $n\times m$.

Посчитайте для каждой пары штатов, как и в первом задании, достигаемый уровень значимости с помощью точного критерия Фишера и сравните получившиеся значения с двумя другими подходами, описанными выше.

Точный критерий Фишера удобно вычислять с помощью функции 
```
scipy.stats.fisher_exact
```
которая принимает на вход таблицу сопряженности 2x2. 

In [8]:
states_combs = combinations(states, 2)
states_churn_p = []
for comb in states_combs:
    subtable = states_crosstab.loc[comb,:]
    states_churn_p.append(stats.fisher_exact(subtable)[1])
alpha = 0.05
states_churn_p = np.array(states_churn_p)
print('Среднее ',states_churn_p.mean())
print('У стольких пар значения pvalue меньше', alpha, ':', sum(states_churn_p < alpha))

Среднее  0.6483383060020681
У стольких пар значения pvalue меньше 0.05 : 10


- ~~Точный критерий Фишера точно также, как и критерий xи-квадрат, нельзя использовать, если наблюдений < 40 и если ожидаемое значение меньше 5 больше чем в 20% ячейках.~~

- Точный критерий Фишера всегда лучше, чем критерий xи-квадрат, потому что не использует аппроксимацию дискретного распределения непрерывным. Однако при увеличении размера выборки его преимущества по сравнению с критерем xи-квадрат уменьшаются, в пределе достигая нуля.

- Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем значительно большие, чем xи-квадрат без поправки

- ~~Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем меньшие, чем xи-квадрат без поправки~~

- ~~Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем большие, чем xи-квадрат с поправкой Йетса~~

- Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем меньшие, чем xи-квадрат с поправкой Йетса

### Корреляция Пирсона
Давайте попробуем применить полученные знания о разных видах корреляции и ее применимости на практике.

Рассмотрим пару признаков day_calls и mes_estim. Посчитайте корреляцию Пирсона между этими признаками на всех данных, ее значимость.

Отметьте все верные утверждения. 

In [9]:
df[['day_calls','mes_estim']].head()

,day_calls,mes_estim
0,110,0.65
1,123,0.55
2,114,0.72
3,71,0.28
4,113,0.45


In [10]:
pearsonr(df.day_calls, df.mes_estim)

(-0.051794350587572625, 0.0027798836869756707)

Корреляции как таковой между этими признаками нет. Поэтому и такой маленький p-value.

- Корреляция Пирсона имеет отрицательный знак, и отличие корреляции от нуля на уровне доверия 0.05 значимо.

- ~~Все варианты неверны, потому что значимость корреляции Пирсона можно оценивать только для нормального распределения, как и упоминалось в лекциях.~~

- ~~Корреляция Пирсона имеет положительный знак, и отличие корреляции от нуля на уровне доверия 0.05 не значимо.~~

- ~~Корреляция Пирсона имеет отрицательный знак, и отличие корреляции от нуля на уровне доверия 0.05 не значимо.~~

- ~~Корреляция Пирсона имеет положительный знак, и отличие корреляции от нуля на уровне доверия 0.05 значимо.~~

### Корреляция Спирмена
Еще раз рассмотрим пару признаков day_calls и mes_estim. Посчитайте корреляцию Спирмена между этими признаками на всех данных, ее значимость.

Отметьте все верные утверждения. 

In [11]:
spearmanr(df.day_calls, df.mes_estim)

SpearmanrResult(correlation=0.043349880533927444, pvalue=0.012317367189170541)

Корреляции как таковой между этими признаками нет. Поэтому и такой маленький p-value.

- ~~Корреляция Спирмена тут неприменима, поскольку речь идет о непрерывных величинах, а корреляция Спирмена применяется к выборочным рангам двух выборок.~~

- ~~Корреляция Спирмена имеет отрицательный знак, и отличие корреляции от нуля на уровне доверия 0.05 значимо.~~

- ~~Корреляция Спирмена имеет положительный знак, и отличие корреляции от нуля на уровне доверия 0.05 не значимо.~~

- ~~Корреляция Спирмена имеет отрицательный знак, и отличие корреляции от нуля на уровне доверия 0.05 не значимо.~~

- Корреляция Спирмена имеет положительный знак, и отличие корреляции от нуля на уровне доверия 0.05 значимо.

### Интерпретация корреляций
Как можно интерпретировать полученные значения коэффициентов корреляции и достигаемые уровни значимости при проверки гипотез о равенстве нулю этих коэффициентов?
- ~~Не стоит ориентироваться на значение корреляции Спирмена, потому что корреляцию Спирмена можно считать только тогда, когда оба признака дискретные и между значениями можно установить строгий порядок.~~

- ~~Предположение нормальности данных двух признаков не выполнено, что хорошо видно на ку-ку графике, поэтому корреляция Пирсона может быть полностью неадекватна.~~

- Посчитанные корреляции и их значимости говорят лишь о том, что необходимо взглянуть на данные глазами и попытаться понять, что приводит к таким (противоречивым?) результатам.

- ~~Подсчет корреляций не имеет особого смысла, поскольку корреляция ничего не говорит о том, какая на самом деле зависимость имеется между признаками.~~

### Корреляция Крамера
Посчитайте значение коэффицента корреляции Крамера между двумя признаками: штатом (state) и оттоком пользователей (churn) для всех пользователей, которые находились в контрольной группе (treatment=1). Что можно сказать о достигаемом уровне значимости при проверке гипотезы о равенство нулю этого коэффициента?

In [12]:
df_treatment_1[['state', 'churn']].head()

,state,churn
0,KS,False.
3,OH,False.
8,LA,False.
12,IA,False.
17,VT,False.


In [13]:
crosstab_state_churn = pd.crosstab(df_treatment_1.state, df_treatment_1.churn)
chi2, p, _, _ = stats.chi2_contingency(crosstab_state_churn)
print('Достигаемый уровень значимости равен', p)

Достигаемый уровень значимости равен 0.7097590042778473


- ~~Достигаемый уровень значимости < 0.05, то есть, отличие от нуля значения коэффицента Крамера значимо.~~

- ~~Достигаемый уровень значимости > 0.05, то есть, отличие от нуля значения коэффицента Крамера незначимо.~~

- Для вычисления коэффициента Крамера используется значение статистики xи-квадрат, на которую мы не можем положиться применительно к нашим данным.

- ~~Коэффициент корреляции Крамера не может быть использован для сравнения связи этих двух признаков, потому что он используется для таблиц сопряженности, где каждая из размерностей больше двух. Если хотя бы одна из размерностей равна 2, то нужно использовать коэффициент корреляции Мэтьюса.~~

### Самостоятельность
Вы прослушали большой курс и к текущему моменту обладете достаточными знаниями, чтобы попытаться самостоятельно выбрать нужный метод / инструмент / статистический критерий и сделать правильное заключение.

В этой части задания вам нужно будет самостоятельно решить, с помощью каких методов можно провести анализ эффективности удержания (churn) с помощью раличных методов (treatment = 0, treatment = 2) относительно контрольной группы пользователей (treatment = 1).

Что можно сказать об этих двух методах (treatment = 0, treatment = 2)? Одинаковы ли они с точки зрения эффективности? Каким бы методом вы бы посоветовали воспользоваться компании?

Не забудьте про поправку на множественную проверку! И не пользуйтесь односторонними альтернативами, поскольку вы не знаете, к каким действительно последствиям приводят тестируемые методы (treatment = 0, treatment = 2) !

#### Критерий Манна-Уитни

In [14]:
treatments = df.treatment.unique()
df_treat_p_vals = pd.DataFrame(list(combinations(treatments, 2)), columns = ['treat_1', 'treat_2'])
df_treat_p_vals['p'] = 0.
for i in np.arange(df_treat_p_vals.shape[0]):
    stat, p = stats.mannwhitneyu(df[df.treatment == df_treat_p_vals.loc[i][0]].churn, 
                                 df[df.treatment == df_treat_p_vals.loc[i][1]].churn)
    df_treat_p_vals.at[i,'p'] = p  

In [15]:
reject, p_corrected, a1, a2 = multipletests(df_treat_p_vals.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 
df_treat_p_vals['p_corrected'] = p_corrected
df_treat_p_vals['reject'] = reject

In [16]:
df_treat_p_vals

,treat_1,treat_2,p,p_corrected,reject
0,1,0,0.114228,0.156534,False
1,1,2,0.004683,0.014049,True
2,0,2,0.078267,0.156534,False


- ~~Ни один из методов не показал значительного улучшения относительно других, о чем говорит групповой статистический критерий.~~

- ~~treatment = 0 статистически значимо отличается от контрольной группы treatment = 1~~

- treatment = 2 статистически значимо отличается от контрольной группы treatment = 1

- ~~В дальнейшем телеком компании рекомендуется использовать и treatment = 0, и treatment = 2 для наибольшей эффективности удержения абонентов.~~

- Отличие между treatment = 0 и treatment = 2 относительно влияния на уровень churn статистически незначимо.

## Вывод


Повторил корреляцию и требования к данным для корректности результатов.